# ETL

Este proceso de ETL se concentra únicamente en extraer y transformar los datos necesarios en relación a la población de CABA para los años de 2016 a 2021.

In [12]:
# Importacion de librerias
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [13]:
# Funcion para realizar web-scrapping

def get_population_data():
    """
    Extrae los datos de población de CABA para los años 2010, 2015 y 2020 mediante web scraping.

    Devuelve:
        Una lista de tuplas con los siguientes datos:
            * Jurisdicción
            * Población en el año 2010
            * Población en el año 2015
            * Población en el año 2020
    """

    url = "https://www.ign.gob.ar/NuestrasActividades/Geografia/DatosArgentina/Poblacion2"
    response = requests.get(url)
    soup = BeautifulSoup(response.content, "html.parser")

    table = soup.find("table", class_="table table-striped pob2020")
    rows = table.find_all("tr")

    data = []

    # Iterar a través de las filas (tr) después de la primera fila (encabezados)
    for row in rows[1:]:
        cells = row.find_all(["th", "td"])
        jurisdiccion = cells[0].text
        poblacion_2010 = cells[1].text
        poblacion_2015 = cells[2].text
        poblacion_2020 = cells[3].text
        data.append((jurisdiccion, poblacion_2010, poblacion_2015, poblacion_2020))

    # Filtrar solo el registro que contiene 'Ciudad Autónoma de Buenos Aires' en la columna '0'
    filtered_data = [row for row in data if 'Ciudad Autónoma de Buenos Aires' in row[0]]

    return filtered_data

In [14]:
# Aplicacion de la Funcion get_population_data
data = get_population_data()
data

[('Ciudad Autónoma de Buenos Aires', '2.890.151', '3.054.267', '3.075.646')]

In [15]:
# Convertir la lista de tuplas a un DataFrame
poblacion = pd.DataFrame(data, columns=['Jurisdicción', '2010', '2015', '2020'])

# Eliminar los puntos de miles en las columnas numéricas
poblacion[['2010', '2015', '2020']] = poblacion[['2010', '2015', '2020']].apply(lambda x: x.str.replace('.', '').astype(int))

# Configurar la columna 'Jurisdicción' como índice
poblacion.set_index('Jurisdicción', inplace=True)

# Transponer el DataFrame para intercambiar filas y columnas
poblacion = poblacion.transpose()

# Renombrar las columnas
poblacion.columns = ['Población estimada']

# Restablecer el índice para obtener las columnas 'Año' y 'Población estimada'
poblacion.reset_index(inplace=True)

# Renombrar la columna 'index' a 'Año'
poblacion.rename(columns={'index': 'Año'}, inplace=True)

# Eliminar registros con 'Año' igual a 2010 
poblacion = poblacion[~poblacion['Año'].isin(['2010'])]

# Mostrar el DataFrame resultante
print(poblacion)


    Año  Población estimada
1  2015             3054267
2  2020             3075646


In [16]:
poblacion['Año'] = pd.to_numeric(poblacion['Año'])

poblacion.info()

In [18]:
# Agregar años adicionales
df_nuevos_años = pd.DataFrame({'Año': [2016, 2017, 2018, 2019, 2021]}, dtype=int)

# Unir los DataFrames 
poblacion = pd.concat([poblacion, df_nuevos_años], ignore_index=True)

# Reordenar el DataFrame según los valores de la columna 'Año' en orden ascendente
poblacion = poblacion.sort_values(by='Año', ascending=True).reset_index(drop=True)

poblacion

,Año,Población estimada
0,2015,3054267.0
1,2016,NaN
2,2017,NaN
3,2018,NaN
4,2019,NaN
5,2020,3075646.0
6,2021,NaN


In [19]:
# Calcular población estimada asumiendo un crecimiento lineal para los años 2016 a 2019
poblacion['Población estimada'] = pd.to_numeric(poblacion['Población estimada'])
pendiente = (poblacion.iloc[-2, 1] - poblacion.iloc[0, 1]) / (len(poblacion) - 2)

# Asignar valores para los años 2016 a 2019
for i in range(1, len(poblacion) - 1):
    poblacion.at[i, 'Población estimada'] = round(poblacion.iloc[0, 1] + pendiente * (poblacion.iloc[i, 0] - poblacion.iloc[0, 0]))

# Calcular población estimada para el año 2021
poblacion.iloc[-1, 1] = round(poblacion.iloc[-2, 1] + pendiente)

poblacion['Población estimada'] = poblacion['Población estimada'].astype(int)

# Eliminar registros con 'Año' igual a 2015 
poblacion = poblacion[~poblacion['Año'].isin([2015])]

# DataFrame resultante
poblacion


,Año,Población estimada
1,2016,3058543
2,2017,3062819
3,2018,3067094
4,2019,3071370
5,2020,3075646
6,2021,3079922


In [20]:
# Exportacion datasets como archivo .csv
poblacion.to_csv('datasets/procesados/poblacion_estimada', index=False)